# Import Required Libraries

In [ ]:
import numpy as np
import pandas as pd
import datetime
from collections import Counter
import os
import pickle
import io

import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("whitegrid")
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.float_format", lambda x: "%.2f" % x)

from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.linear_model import Ridge, Lasso, LinearRegression, ElasticNet
from sklearn.svm import SVR, LinearSVR
from sklearn.kernel_ridge import KernelRidge
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.feature_selection import f_regression
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import SelectKBest

from IPython.display import Markdown

def bold(string):
    display(Markdown("**" + string + "**"))

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

# Load Data

In [ ]:
df_ = pd.read_csv("/mnt/hdd/Datasets/TrendyolSmartPhone.csv")
df = df_.copy()
df.head()

In [ ]:
def df_stats(data):
    bold("**" + " SHAPE ".center(50, "#") + "**")
    print("ROWS: {}".format(data.shape[0]))
    print("COLUMNS: {}".format(data.shape[1]))
    bold("**" + " TYPES ".center(50, "#") + "**")
    print(data.dtypes)
    bold("**" + " MISSING VALUES ".center(50, "#") + "**")
    print(data.isnull().sum())
    bold("**" + " DUPLICATED VALUES ".center(50, "#") + "**")
    print("NUMBER OF DUPLICATED VALUES: {}".format(data.duplicated().sum()))
    bold("**" + " MEMORY USAGE ".center(50, "#") + "**")
    buf = io.StringIO()
    data.info(buf=buf)
    info = buf.getvalue().split("\n")[-2].split(":")[1].strip()
    print("Memory Usage: {}".format(info))
    bold("**" + " DESCRIBE ".center(50, "#") + "**")
    print(data.describe().T)

In [ ]:
df_stats(df)

In [ ]:
df.drop(["Urun Başlığı", "Link"], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

# Fill NaN

In [ ]:
df["Görüntü Teknolojisi"].value_counts()

In [ ]:
df["Görüntü Teknolojisi"].fillna("LCD", inplace=True)

In [ ]:
df["Ekran Çözünürlüğü"].value_counts()

In [ ]:
df["Ekran Çözünürlüğü"].fillna("HD", inplace=True)

In [ ]:
df["Kamera Çözünürlüğü"].value_counts()

In [ ]:
df["Kamera Çözünürlüğü"].fillna("20 MP ve üstü", inplace=True)

In [ ]:
df["Batarya Kapasitesi Aralığı"].value_counts()

In [ ]:
df["Batarya Kapasitesi Aralığı"].fillna("3500-4000 mAh", inplace=True)

In [ ]:
df["Mobil Bağlantı Hızı"].value_counts()

In [ ]:
df["Mobil Bağlantı Hızı"].fillna("4.5G", inplace=True)

In [ ]:
df["Ekran Boyutu"].value_counts()

In [ ]:
df["Ekran Boyutu"].fillna("6 inç ve üstü", inplace=True)

In [ ]:
df["Parmak İzi Okuyucu"].value_counts()

In [ ]:
df["Parmak İzi Okuyucu"].fillna("Var", inplace=True)

In [ ]:
df["Yüz Tanıma"].value_counts()

In [ ]:
df["Yüz Tanıma"].fillna("Var", inplace=True)

In [ ]:
df["Suya/Toza Dayanıklılık"].value_counts()

In [ ]:
df["Suya/Toza Dayanıklılık"].fillna("Var", inplace=True)

In [ ]:
df["Çift Hat"].value_counts()

In [ ]:
df["Çift Hat"].fillna("Var", inplace=True)

In [ ]:
df["Arttırılabilir Hafıza (Hafıza Kartı Desteği)"].value_counts()

In [ ]:
df["Arttırılabilir Hafıza (Hafıza Kartı Desteği)"].fillna("Var", inplace=True)

In [ ]:
df["Ekran Yenileme Hızı"].value_counts()

In [ ]:
df["Ekran Yenileme Hızı"].fillna("60 Hz", inplace=True)

In [ ]:
df.shape

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

# EDA

In [ ]:
df["Marka"].value_counts().plot(kind="bar")

In [ ]:
df["İşletim Sistemi"].value_counts().plot(kind="bar")

In [ ]:
df["Dahili Hafıza"].value_counts().plot(kind="bar")

In [ ]:
df["RAM Kapasitesi"].value_counts().plot(kind="bar")

In [ ]:
df['Görüntü Teknolojisi'].value_counts().plot(kind="bar")

In [ ]:
df['Ekran Çözünürlüğü'].value_counts().plot(kind="bar")

In [ ]:
df['Kamera Çözünürlüğü'].value_counts().plot(kind="bar")

In [ ]:
df['Batarya Kapasitesi Aralığı'].value_counts().plot(kind="bar")

In [ ]:
df['Mobil Bağlantı Hızı'].value_counts().plot(kind="bar")

In [ ]:
df['Ekran Boyutu'].value_counts().plot(kind="bar")

In [ ]:
df['Parmak İzi Okuyucu'].value_counts().plot(kind="bar")

In [ ]:
df['Yüz Tanıma'].value_counts().plot(kind="bar")

In [ ]:
df['Suya/Toza Dayanıklılık'].value_counts().plot(kind="bar")

In [ ]:
df['Çift Hat'].value_counts().plot(kind="bar")

In [ ]:
df['Arttırılabilir Hafıza (Hafıza Kartı Desteği)'].value_counts().plot(kind="bar")

In [ ]:
df['Ekran Yenileme Hızı'].value_counts().plot(kind="bar")

# Preprocess

In [ ]:
df.head()

In [ ]:
df.drop(df[df["Dahili Hafıza"] == "8 GB"].index, inplace = True)
df.drop(df[df["Dahili Hafıza"] == "4 GB"].index, inplace = True)
df.drop(df[df["Dahili Hafıza"] == "512 MB"].index, inplace = True)
df.drop(df[df["Dahili Hafıza"] == "Yok"].index, inplace = True)

In [ ]:
df.drop(df[df["RAM Kapasitesi"] == "32 GB"].index, inplace = True)

In [ ]:
df["Görüntü Teknolojisi"] = df["Görüntü Teknolojisi"].map({"LCD": "LCD",
                                                           "AMOLED": "AMOLED",
                                                           "OLED": "OLED",
                                                           "LED": "AMOLED"})

In [ ]:
df["Ekran Çözünürlüğü"] = df["Ekran Çözünürlüğü"].map({"HD": "HD",
                                                       "FHD+": "FHD",
                                                       "HD+": "HD",
                                                       "QHD+": "QHD",
                                                       "QHD": "QHD",
                                                       "UHD+": "UHD+"})

In [ ]:
df.drop(df[df["Ekran Çözünürlüğü"] == "UHD+"].index, inplace = True)

In [ ]:
df["Kamera Çözünürlüğü"] = df["Kamera Çözünürlüğü"].map({"20 MP ve üstü": "15 MP ve üstü",
                                                         "10 - 15 MP": "15 MP ve altı",
                                                         "15 - 20 MP": "15 MP ve üstü",
                                                         "5 - 10 MP": "15 MP ve altı",
                                                         "5 MP ve altı": "15 MP ve altı"})

In [ ]:
df.drop(df[df["Ekran Boyutu"] == "65\" / 165 Ekran"].index, inplace = True)  
df.drop(df[df["Ekran Boyutu"] == "24\" / 61 Ekran"].index, inplace = True)
df.drop(df[df["Ekran Boyutu"] == "7 - 10 inç"].index, inplace = True)

In [ ]:
df["Ekran Boyutu"] = df["Ekran Boyutu"].map({'6 inç\xa0ve üstü': "6 inç ve üstü",
                                             "6 inç ve üstü": "6 inç ve üstü",
                                             "6 inch ve üstü": "6 inç ve üstü",
                                             "6 - 8 inç": "6 inç ve üstü",
                                             "4 - 4,5 inç": "6 inç ve altı",
                                             "6 inç ve altı": "6 inç ve altı",
                                             "5 - 5,5 inç": "6 inç ve altı",
                                             '5,5 - 6 inç': "6 inç ve altı",
                                             '6,8 inç': "6 inç ve üstü",
                                             '10,5 inç': "6 inç ve üstü",
                                             '4,5 - 5 inç': "6 inç ve altı"})

In [ ]:
df["Ekran Yenileme Hızı"] = df["Ekran Yenileme Hızı"].map({"60 Hz": "60 Hz",
                                                           "120 Hz": "60 Hz üstü",
                                                           "90 Hz": "60 Hz üstü",
                                                           "60 Hz altı": "60 Hz altı",
                                                           "144 Hz": "60 Hz üstü",
                                                           "144 Hz üstü": "60 Hz üstü"})

In [ ]:
df.head()

In [ ]:
df.to_csv("preprocessed.csv")

In [ ]:
df["Fiyat"] = df["Fiyat"].apply(lambda x: x.replace(" TL", ""))

In [ ]:
df["Fiyat"] = df["Fiyat"].astype(int)

# Outlier Detection

In [ ]:
def custom_boxplot(df, columns, title):
    fig, ax = plt.subplots(1 , 1, sharey=True, figsize=(15, len(columns) * 2.5))
    fig.suptitle(title, y=1, size=25)
    
    for i, column in enumerate(columns):
        sns.boxplot(data=df[column], orient="h", ax=ax)
        ax.set_title(column + ", skewness is: " + str(round(df[column].skew(axis=0, skipna=True), 2)))

    plt.tight_layout()
    plt.show()

In [ ]:
custom_boxplot(df, ["Fiyat"], "Boxplots for each numeric variable")

# Feature Scaling

In [ ]:
X = df.drop("Fiyat", axis=1)
y = df["Fiyat"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

In [ ]:
encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)
X_train["Marka"] = encoder.fit_transform(X_train["Marka"].values.reshape(-1, 1))
X_test["Marka"] = encoder.transform(X_test["Marka"].values.reshape(-1, 1))[:, 0]

X_train["İşletim Sistemi"] = encoder.fit_transform(X_train["İşletim Sistemi"].values.reshape(-1, 1))
X_test["İşletim Sistemi"] = encoder.transform(X_test["İşletim Sistemi"].values.reshape(-1, 1))

X_train["Dahili Hafıza"] = encoder.fit_transform(X_train["Dahili Hafıza"].values.reshape(-1, 1))
X_test["Dahili Hafıza"] = encoder.transform(X_test["Dahili Hafıza"].values.reshape(-1, 1))

X_train["RAM Kapasitesi"] = encoder.fit_transform(X_train["RAM Kapasitesi"].values.reshape(-1, 1))
X_test["RAM Kapasitesi"] = encoder.transform(X_test["RAM Kapasitesi"].values.reshape(-1, 1))

X_train["Görüntü Teknolojisi"] = encoder.fit_transform(X_train["Görüntü Teknolojisi"].values.reshape(-1, 1))
X_test["Görüntü Teknolojisi"] = encoder.transform(X_test["Görüntü Teknolojisi"].values.reshape(-1, 1))

X_train["Ekran Çözünürlüğü"] = encoder.fit_transform(X_train["Ekran Çözünürlüğü"].values.reshape(-1, 1))
X_test["Ekran Çözünürlüğü"] = encoder.transform(X_test["Ekran Çözünürlüğü"].values.reshape(-1, 1))

X_train["Kamera Çözünürlüğü"] = encoder.fit_transform(X_train["Kamera Çözünürlüğü"].values.reshape(-1, 1))
X_test["Kamera Çözünürlüğü"] = encoder.transform(X_test["Kamera Çözünürlüğü"].values.reshape(-1, 1))

X_train["Batarya Kapasitesi Aralığı"] = encoder.fit_transform(X_train["Batarya Kapasitesi Aralığı"].values.reshape(-1, 1))
X_test["Batarya Kapasitesi Aralığı"] = encoder.transform(X_test["Batarya Kapasitesi Aralığı"].values.reshape(-1, 1))

X_train["Mobil Bağlantı Hızı"] = encoder.fit_transform(X_train["Mobil Bağlantı Hızı"].values.reshape(-1, 1))
X_test["Mobil Bağlantı Hızı"] = encoder.transform(X_test["Mobil Bağlantı Hızı"].values.reshape(-1, 1))

X_train["Ekran Boyutu"] = encoder.fit_transform(X_train["Ekran Boyutu"].values.reshape(-1, 1))
X_test["Ekran Boyutu"] = encoder.transform(X_test["Ekran Boyutu"].values.reshape(-1, 1))

X_train["Parmak İzi Okuyucu"] = encoder.fit_transform(X_train["Parmak İzi Okuyucu"].values.reshape(-1, 1))
X_test["Parmak İzi Okuyucu"] = encoder.transform(X_test["Parmak İzi Okuyucu"].values.reshape(-1, 1))
 
X_train["Yüz Tanıma"] = encoder.fit_transform(X_train["Yüz Tanıma"].values.reshape(-1, 1))
X_test["Yüz Tanıma"] = encoder.transform(X_test["Yüz Tanıma"].values.reshape(-1, 1))

X_train["Suya/Toza Dayanıklılık"] = encoder.fit_transform(X_train["Suya/Toza Dayanıklılık"].values.reshape(-1, 1))
X_test["Suya/Toza Dayanıklılık"] = encoder.transform(X_test["Suya/Toza Dayanıklılık"].values.reshape(-1, 1))

X_train["Çift Hat"] = encoder.fit_transform(X_train["Çift Hat"].values.reshape(-1, 1))
X_test["Çift Hat"] = encoder.transform(X_test["Çift Hat"].values.reshape(-1, 1))

X_train["Arttırılabilir Hafıza (Hafıza Kartı Desteği)"] = encoder.fit_transform(X_train["Arttırılabilir Hafıza (Hafıza Kartı Desteği)"].values.reshape(-1, 1))
X_test["Arttırılabilir Hafıza (Hafıza Kartı Desteği)"] = encoder.transform(X_test["Arttırılabilir Hafıza (Hafıza Kartı Desteği)"].values.reshape(-1, 1))

X_train["Ekran Yenileme Hızı"] = encoder.fit_transform(X_train["Ekran Yenileme Hızı"].values.reshape(-1, 1))
X_test["Ekran Yenileme Hızı"] = encoder.transform(X_test["Ekran Yenileme Hızı"].values.reshape(-1, 1))

In [ ]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

# Model

In [ ]:
def adjusted_r2(r2, n, k):
    return r2 - (k - 1) / (n - k) * (1 - r2)

In [ ]:
def root_mse(y_test, y_pred):
    return np.sqrt(mean_squared_error(y_test, y_pred))

In [ ]:
kf = KFold(n_splits=5, shuffle=False)

In [ ]:
scores_df = pd.DataFrame({
    "Model": [],
    "R2": [],
    "Adjusted R2": [],
    "MSE": [],
    "RMSE": [],
    "MAE": []
})

In [ ]:
lr = LinearRegression()
ridge = Ridge()
lasso = Lasso()
en = ElasticNet()
svr = SVR()
rf = RandomForestRegressor()
gb = GradientBoostingRegressor()
ada = AdaBoostRegressor()
xgb = XGBRegressor()
lgbm = LGBMRegressor()
knn = KNeighborsRegressor()

In [ ]:
def add_to_scores_df(log, model_name, idx):
    scores_df.loc[idx] = [model_name, log["R2"][0], log["Adjusted R2"][0], log["MSE"][0], log["RMSE"][0], log["MAE"][0]]

In [ ]:
def train_model(model):
    model_log = pd.DataFrame(columns=["R2", "Adjusted R2", "MSE", "RMSE", "MAE"])
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    model_log.loc[0, "R2"] = r2_score(y_test, y_pred)
    model_log.loc[0, "Adjusted R2"] = adjusted_r2(r2_score(y_test, y_pred), X.shape[0], X.shape[1])
    model_log.loc[0, "MSE"] = mean_squared_error(y_test, y_pred)
    model_log.loc[0, "RMSE"] = root_mse(y_test, y_pred)
    model_log.loc[0, "MAE"] = mean_absolute_error(y_test, y_pred)

    fig, ax = plt.subplots(figsize=(8, 6))
    sns.regplot(x=y_test, y=y_pred)
    plt.show()

    return model_log

In [ ]:
lr_log = train_model(lr)
add_to_scores_df(lr_log, "Linear Regression", 0)
lr_log

In [ ]:
ridge_log = train_model(ridge)
add_to_scores_df(ridge_log, "Ridge", 1)
ridge_log

In [ ]:
lasso_log = train_model(lasso)
add_to_scores_df(lasso_log, "Lasso", 2)
lasso_log

In [ ]:
en_log = train_model(en)
add_to_scores_df(en_log, "ElasticNet", 3)
en_log

In [ ]:
rf_log = train_model(rf)
add_to_scores_df(rf_log, "Random Forests", 4)
rf_log

In [ ]:
gb_log = train_model(gb)
add_to_scores_df(gb_log, "Gradient Boosting", 5)
gb_log

In [ ]:
ada_log = train_model(ada)
add_to_scores_df(ada_log, "Ada Boost", 6)
ada_log

In [ ]:
svr_log = train_model(svr)
add_to_scores_df(svr_log, "SVR", 7)
svr_log

In [ ]:
lgbm_log = train_model(lgbm)
add_to_scores_df(lgbm_log, "LGBM", 8)
lgbm_log

In [ ]:
xgb_log = train_model(xgb)
add_to_scores_df(xgb_log, "XGBoost", 9)
xgb_log

In [ ]:
knn_log = train_model(knn)
add_to_scores_df(knn_log, "KNeighbors", 10)
knn_log

In [ ]:
scores_df

In [ ]:
plt.figure(figsize=(10, 20))

plt.subplot(511)
ax = sns.barplot(data=scores_df.sort_values(by="R2", ascending=False), y="Model", x="R2")
for container in ax.containers:
    ax.bar_label(container)
plt.title("Model / R2 Score")
plt.xlabel("")

plt.subplot(512)
ax = sns.barplot(data=scores_df.sort_values(by="Adjusted R2", ascending=False), y="Model", x="Adjusted R2")
for container in ax.containers:
    ax.bar_label(container)
plt.title("Model / Adjusted R2")
plt.xlabel("")

plt.subplot(513)
ax = sns.barplot(data=scores_df.sort_values(by="MSE", ascending=True), y="Model", x="MSE")
for container in ax.containers:
    ax.bar_label(container)
plt.title("Model / MSE")
plt.xlabel("")

plt.subplot(514)
ax = sns.barplot(data=scores_df.sort_values(by="RMSE", ascending=True), y="Model", x="RMSE")
for container in ax.containers:
    ax.bar_label(container)
plt.title("Model / RMSE")
plt.xlabel("")

plt.subplot(515)
ax = sns.barplot(data=scores_df.sort_values(by="MAE", ascending=True), y="Model", x="MAE")
for container in ax.containers:
    ax.bar_label(container)
plt.title("Model / MAE")
plt.xlabel("")

plt.show()

# Feature Selection

In [ ]:
def plot_correlation_table(df):
    corr = df.corr()
    fig, axes = plt.subplots(figsize=(20, 20))
    mask = np.triu(np.ones_like(corr))
    sns.heatmap(corr, mask=mask, linewidth=0.6, annot=True, robust=True, center=0, square=True)
    plt.title("Correlation Table")
    plt.show()

In [ ]:
plot_correlation_table(pd.DataFrame(X_train, columns=X.columns))

In [ ]:
f_value = f_regression(X_train, y_train)

for feature in zip(X.columns, f_value[0]):
    print(feature)

In [ ]:
plt.figure(figsize=(4, 4))
plt.barh(y=X.columns, width=f_value[0])
plt.xticks(rotation="vertical")
plt.xlabel("F-value")
plt.title("F-value Comparison")
plt.show()

In [ ]:
MI_score = mutual_info_regression(X_train, y_train, random_state=42)

for feature in zip(X.columns, MI_score):
    print(feature)

In [ ]:
plt.figure(figsize=(6, 6))
plt.barh(y=X.columns, width=MI_score)
plt.xticks(rotation="vertical")
plt.yticks(rotation="horizontal")
plt.xlabel("Mutual Information Score")
plt.title("Mutual Information Score Comparison")
plt.show()

In [ ]:
skb = SelectKBest(score_func=f_regression, k=12)                  
X_data_new = skb.fit_transform(X_train, y_train)

print('Number of features before feature selection: {}'.format(X_train.shape[1]))
print('Number of features after feature selection: {}'.format(X_data_new.shape[1]))

for feature_list_index in skb.get_support(indices=True):
    print('- ' + X.columns[feature_list_index])

In [ ]:
X_train_new = pd.DataFrame(X_train, columns=X.columns)
X_test_new = pd.DataFrame(X_test, columns=X.columns)

In [ ]:
selected_cols = ["Marka", "İşletim Sistemi", "RAM Kapasitesi",
                 "Görüntü Teknolojisi", "Batarya Kapasitesi Aralığı", "Mobil Bağlantı Hızı", "Ekran Boyutu",
                 "Parmak İzi Okuyucu", "Yüz Tanıma", "Suya/Toza Dayanıklılık", "Çift Hat", "Arttırılabilir Hafıza (Hafıza Kartı Desteği)"]
X_train_new = X_train_new[selected_cols]
X_test_new = X_test_new[selected_cols]

In [ ]:
f_scores_df = pd.DataFrame({
    "Model": [],
    "R2": [],
    "Adjusted R2": [],
    "MSE": [],
    "RMSE": [],
    "MAE": []
})

In [ ]:
f_lr = LinearRegression()
f_ridge = Ridge()
f_lasso = Lasso()
f_en = ElasticNet()
f_svr = SVR()
f_rf = RandomForestRegressor()
f_gb = GradientBoostingRegressor()
f_ada = AdaBoostRegressor()
f_xgb = XGBRegressor()
f_lgbm = LGBMRegressor()
f_knn = KNeighborsRegressor()

In [ ]:
def add_to_f_scores_df(log, model_name, idx):
    f_scores_df.loc[idx] = [model_name, log["R2"][0], log["Adjusted R2"][0], log["MSE"][0], log["RMSE"][0], log["MAE"][0]]

In [ ]:
def train_model(model):
    model_log = pd.DataFrame(columns=["R2", "Adjusted R2", "MSE", "RMSE", "MAE"])
    model.fit(X_train_new, y_train)

    y_pred = model.predict(X_test_new)

    model_log.loc[0, "R2"] = r2_score(y_test, y_pred)
    model_log.loc[0, "Adjusted R2"] = adjusted_r2(r2_score(y_test, y_pred), X_train_new.shape[0], X_train_new.shape[1])
    model_log.loc[0, "MSE"] = mean_squared_error(y_test, y_pred)
    model_log.loc[0, "RMSE"] = root_mse(y_test, y_pred)
    model_log.loc[0, "MAE"] = mean_absolute_error(y_test, y_pred)

    fig, ax = plt.subplots(figsize=(8, 6))
    sns.regplot(x=y_test, y=y_pred)
    plt.show()

    return model_log

In [ ]:
f_lr_log = train_model(f_lr)
add_to_f_scores_df(f_lr_log, "Linear Regression", 0)
f_lr_log

In [ ]:
f_ridge_log = train_model(f_ridge)
add_to_f_scores_df(f_ridge_log, "Ridge", 1)
f_ridge_log

In [ ]:
f_lasso_log = train_model(f_lasso)
add_to_f_scores_df(f_lasso_log, "Lasso", 2)
f_lasso_log

In [ ]:
f_en_log = train_model(f_en)
add_to_f_scores_df(f_en_log, "ElasticNet", 3)
f_en_log

In [ ]:
f_rf_log = train_model(f_rf)
add_to_f_scores_df(f_rf_log, "Random Forests", 4)
f_rf_log

In [ ]:
f_gb_log = train_model(f_gb)
add_to_f_scores_df(f_gb_log, "Gradient Boosting", 5)
f_gb_log

In [ ]:
f_ada_log = train_model(f_ada)
add_to_f_scores_df(f_ada_log, "Ada Boost", 6)
f_ada_log

In [ ]:
f_svr_log = train_model(f_svr)
add_to_f_scores_df(f_svr_log, "SVR", 7)
f_svr_log

In [ ]:
f_lgbm_log = train_model(f_lgbm)
add_to_f_scores_df(f_lgbm_log, "LGBM", 8)
f_lgbm_log

In [ ]:
f_xgb_log = train_model(f_xgb)
add_to_f_scores_df(f_xgb_log, "XGBoost", 9)
f_xgb_log

In [ ]:
f_knn_log = train_model(f_knn)
add_to_f_scores_df(f_knn_log, "KNeighbors", 10)
f_knn_log

In [ ]:
plt.figure(figsize=(10, 20))

plt.subplot(511)
ax = sns.barplot(data=f_scores_df.sort_values(by="R2", ascending=False), y="Model", x="R2")
for container in ax.containers:
    ax.bar_label(container)
plt.title("Model / R2 Score")
plt.xlabel("")

plt.subplot(512)
ax = sns.barplot(data=f_scores_df.sort_values(by="Adjusted R2", ascending=False), y="Model", x="Adjusted R2")
for container in ax.containers:
    ax.bar_label(container)
plt.title("Model / Adjusted R2")
plt.xlabel("")

plt.subplot(513)
ax = sns.barplot(data=f_scores_df.sort_values(by="MSE", ascending=True), y="Model", x="MSE")
for container in ax.containers:
    ax.bar_label(container)
plt.title("Model / MSE")
plt.xlabel("")

plt.subplot(514)
ax = sns.barplot(data=f_scores_df.sort_values(by="RMSE", ascending=True), y="Model", x="RMSE")
for container in ax.containers:
    ax.bar_label(container)
plt.title("Model / RMSE")
plt.xlabel("")

plt.subplot(515)
ax = sns.barplot(data=f_scores_df.sort_values(by="MAE", ascending=True), y="Model", x="MAE")
for container in ax.containers:
    ax.bar_label(container)
plt.title("Model / MAE")
plt.xlabel("")

plt.show()

In [ ]:
scores_df.sort_values(by="R2", ascending=False)

In [ ]:
f_scores_df.sort_values(by="R2", ascending=False)

# Hyperparameter Tuning

In [ ]:
t_scores_df = pd.DataFrame({
    "Model": [],
    "R2": [],
    "Adjusted R2": [],
    "MSE": [],
    "RMSE": [],
    "MAE": []
})

In [ ]:
def add_to_t_scores_df(log, model_name, idx):
    t_scores_df.loc[idx] = [model_name, log["R2"][0], log["Adjusted R2"][0], log["MSE"][0], log["RMSE"][0], log["MAE"][0]]

In [ ]:
rf_params = {
    "bootstrap": [True, False],
    "max_depth": [2, 3, 4, 5, 6, 8],
    "max_features": [0.05, 0.1, 0.2, 0.5, 0.7, 0.9, 0.95, 0.99, 1],
    "min_samples_split": [0.05, 0.1, 0.2, 0.5, 0.7, 0.9, 0.95, 0.99, 1],
    "n_estimators": [50, 100, 250, 500, 1000]
}

rf_model = RandomForestRegressor()

cv = RandomizedSearchCV(rf_model, rf_params, n_iter=10, cv=3, n_jobs=-1, verbose=0)
cv.fit(X_train_new, y_train)

print("Best hyperparams:\n", cv.best_params_)
print("Best score: ", cv.best_score_)

In [ ]:
rf_tuned = RandomForestRegressor(**cv.best_params_)
rf_tuned_log = train_model(rf_tuned)
add_to_t_scores_df(rf_tuned_log, "Random Forests", 0)
rf_tuned_log

In [ ]:
lgbm_params ={
    'learning_rate':[0.01, 0.1, 0.5],
    'n_estimators':[100,500,1000,1250],
    'colsample_bytree': [0.3, 0.7, 1],
    'max_bin':[100, 250, 300],
    'max_depth':[2, 3, 4, 6],
    'min_data_in_leaf':[10, 20, 50],
    'reg_alpha': [1, 2, 5, 10],
    'reg_lambda': [1, 2, 5, 10],
    'min_gain_to_split':[1, 3, 5, 10]
}

lgbm_model = LGBMRegressor()
cv = RandomizedSearchCV(lgbm_model, lgbm_params, n_iter=10, cv=3, n_jobs=-1, verbose=0)
cv.fit(X_train_new, y_train)

print("Best hyperparams:\n", cv.best_params_)
print("Best score: ", cv.best_score_)

In [ ]:
lgbm_tuned = LGBMRegressor(**cv.best_params_)
lgbm_tuned = train_model(lgbm_tuned)
add_to_t_scores_df(lgbm_tuned, "LGBM", 1)
lgbm_tuned

In [ ]:
xgb_params = {
    'n_estimators': [100, 250, 500, 1000],
    'booster':['gbtree', 'gblinear'],
    'learning_rate':[0.01, 0.1, 0.5],
    'max_depth':[2, 3, 5, 7, 10],
    'subsample':[0.5, 0.6, 0.7],
    'min_child_weight': [5, 10, 25],
    'gamma':[5, 10, 25, 50],
    'lambda':[5, 10, 25, 50],
}

xgb_model = XGBRegressor()
cv = RandomizedSearchCV(xgb_model, xgb_params, n_iter=10, cv=3, n_jobs=-1, verbose=0)
cv.fit(X_train_new, y_train)

print("Best hyperparams:\n", cv.best_params_)
print("Best score: ", cv.best_score_)

In [ ]:
xgb_tuned = XGBRegressor(**cv.best_params_)
xgb_tuned_log = train_model(xgb_tuned)
add_to_t_scores_df(xgb_tuned_log, "XGBRegressor", 2)
xgb_tuned_log

# Results

In [ ]:
t_scores_df

In [ ]:
plt.figure(figsize=(10, 20))

plt.subplot(511)
ax = sns.barplot(data=t_scores_df.sort_values(by="R2", ascending=False), y="Model", x="R2")
for container in ax.containers:
    ax.bar_label(container)
plt.title("Model / R2 Score")
plt.xlabel("")

plt.subplot(512)
ax = sns.barplot(data=t_scores_df.sort_values(by="Adjusted R2", ascending=False), y="Model", x="Adjusted R2")
for container in ax.containers:
    ax.bar_label(container)
plt.title("Model / Adjusted R2")
plt.xlabel("")

plt.subplot(513)
ax = sns.barplot(data=t_scores_df.sort_values(by="MSE", ascending=True), y="Model", x="MSE")
for container in ax.containers:
    ax.bar_label(container)
plt.title("Model / MSE")
plt.xlabel("")

plt.subplot(514)
ax = sns.barplot(data=t_scores_df.sort_values(by="RMSE", ascending=True), y="Model", x="RMSE")
for container in ax.containers:
    ax.bar_label(container)
plt.title("Model / RMSE")
plt.xlabel("")

plt.subplot(515)
ax = sns.barplot(data=t_scores_df.sort_values(by="MAE", ascending=True), y="Model", x="MAE")
for container in ax.containers:
    ax.bar_label(container)
plt.title("Model / MAE")
plt.xlabel("")

plt.show()